# MAPS Summer School – Spatial Lag Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vincnardelli/maps/blob/main/Python/notebooks/3_spatial_lag_model_colab.ipynb)

**Data loads automatically from GitHub**

In [1]:
import os
import sys
# Download shapefile data from GitHub
import urllib.request
os.makedirs('data', exist_ok=True)

base_url = 'https://github.com/vincnardelli/maps/raw/main/data/sdr_subnational_data_migration/shps/sdr_subnational_data_dhs_2022_lvl_2'
extensions = ['.shp', '.shx', '.dbf', '.prj']

print('📥 Downloading data from GitHub...')
for ext in extensions:
    url = base_url + ext
    filename = f'data/sdr_subnational_data_dhs_2022_lvl_2{ext}'
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
        print(f'  ✓ {ext}')
    else:
        print(f'  ⊙ {ext} (cached)')

DATA_PATH = 'data/sdr_subnational_data_dhs_2022_lvl_2.shp'
print(f'\n✅ Data ready: {DATA_PATH}')

📥 Downloading data from GitHub...
  ✓ .shp
  ✓ .shx
  ✓ .dbf
  ✓ .prj

✅ Data ready: data/sdr_subnational_data_dhs_2022_lvl_2.shp


In [11]:
# Import required packages
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from libpysal.weights import Queen
from esda.moran import Moran, Moran_Local
from spreg import OLS
from spreg.ml_lag import ML_Lag
from scipy.stats import zscore
from scipy.sparse.csgraph import connected_components
import warnings
warnings.filterwarnings('ignore')

np.random.seed(123)
print('✅ Packages loaded')

✅ Packages loaded


### 2) Read the shapefile

In [3]:
shp_path = DATA_PATH
tz = gpd.read_file(shp_path)

print(f"Shape: {tz.shape}")
print(f"CRS: {tz.crs}")

Shape: (31, 49)
CRS: EPSG:4326


### 3) Variable names & quick summaries

In [4]:
# Dependent variable
vars_y = 'AHMIGRWEMP'  # migration reason: employment

# Independent variables
vars_x = ['EDMDIAWN3M', 'WSSRCEHIMP', 'AHBTHPWIMG', 'EDEDUCWSEH']

print("Variable Summaries:")
print("\n" + "="*60)
for var in [vars_y] + vars_x:
    print(f"\n{var}:")
    print(tz[var].describe())
    print("-"*60)

Variable Summaries:


AHMIGRWEMP:
count    31.000000
mean     12.025806
std       8.667140
min       1.300000
25%       6.200000
50%       8.700000
75%      18.550000
max      31.100000
Name: AHMIGRWEMP, dtype: float64
------------------------------------------------------------

EDMDIAWN3M:
count    31.000000
mean     54.880645
std      15.643346
min      23.400000
25%      48.300000
50%      56.300000
75%      64.950000
max      84.600000
Name: EDMDIAWN3M, dtype: float64
------------------------------------------------------------

WSSRCEHIMP:
count    31.000000
mean     75.461290
std      14.347211
min      49.800000
25%      65.150000
50%      74.600000
75%      83.800000
max      99.500000
Name: WSSRCEHIMP, dtype: float64
------------------------------------------------------------

AHBTHPWIMG:
count    31.000000
mean     33.274194
std      15.166893
min       4.600000
25%      22.050000
50%      37.800000
75%      44.000000
max      58.000000
Name: AHBTHPWIMG, dtype: float64
----

### 5) Baseline OLS regression

In [7]:
# Prepare data for regression
y = tz[vars_y].values.reshape(-1, 1)
X = tz[vars_x].values

# Add constant (intercept)
X_with_const = np.column_stack([np.ones(len(X)), X])
var_names = ['CONSTANT'] + vars_x

# Fit OLS model
ols = OLS(y, X_with_const, name_y=vars_y, name_x=var_names)

print("="*60)
print("OLS REGRESSION RESULTS (PYTHON)")
print("="*60)
print(ols.summary)
print("="*60)

OLS REGRESSION RESULTS (PYTHON)
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :  AHMIGRWEMP                Number of Observations:          31
Mean dependent var  :     12.0258                Number of Variables   :           5
S.D. dependent var  :      8.6671                Degrees of Freedom    :          26
R-squared           :      0.4168
Adjusted R-squared  :      0.3271
Sum squared residual:     1314.19                F-statistic           :      4.6463
Sigma-square        :      50.546                Prob(F-statistic)     :     0.00579
S.E. of regression  :       7.110                Log likelihood        :    -102.065
Sigma-square ML     :      42.393                Akaike info criterion :     214.131
S.E of regression ML:      6.5110                Schwarz criterion     :     221.301

----------------------------

### 6) Spatial weights - Queen contiguity

In [8]:
# Build queen contiguity weights
w_queen = Queen.from_dataframe(tz_geom)
w_queen.transform = 'r'  # row-standardized (equivalent to style="W" in R)

print(f"Number of observations: {w_queen.n}")
print(f"Average number of neighbors: {w_queen.mean_neighbors:.2f}")
print(f"Min neighbors: {min([len(w_queen.neighbors[i]) for i in w_queen.neighbors])}")
print(f"Max neighbors: {max([len(w_queen.neighbors[i]) for i in w_queen.neighbors])}")

Number of observations: 31
Average number of neighbors: 4.00
Min neighbors: 1
Max neighbors: 8


In [9]:
# Get OLS residuals
residuals_ols = ols.u.flatten()

# Moran's I test on residuals
moran_resid = Moran(residuals_ols, w_queen)

print("="*60)
print("MORAN'S I ON OLS RESIDUALS (PYTHON)")
print("="*60)
print(f"Moran's I: {moran_resid.I:.6f}")
print(f"Expected I: {moran_resid.EI:.6f}")
print(f"Variance: {moran_resid.VI_norm:.6e}")
print(f"Z-score: {moran_resid.z_norm:.6f}")
print(f"P-value (two-sided): {moran_resid.p_norm:.6f}")
print("\nInterpretation: Significant spatial autocorrelation in residuals")
print("suggests spatial dependence should be modeled.")
print("="*60)

MORAN'S I ON OLS RESIDUALS (PYTHON)
Moran's I: 0.226981
Expected I: -0.033333
Variance: 1.855839e-02
Z-score: 1.910854
P-value (two-sided): 0.056023

Interpretation: Significant spatial autocorrelation in residuals
suggests spatial dependence should be modeled.


### 8) Fit a Spatial Lag Model (SAR / SLM)

Model specification: y = ρWy + Xβ + ε

The spatial lag model includes a spatially lagged dependent variable (Wy) as an additional regressor.

In [15]:
# Fit Spatial Lag Model using ML estimation
# Note: Using BaseML_Lag directly to avoid summary bug

y_ml = y.reshape(-1, 1)

slm_ml = ML_Lag(y=y_ml, x=X_with_const, w=w_queen, method='full')

print("="*60)
print("SPATIAL LAG MODEL RESULTS (PYTHON - ML)")
print("="*60)
print(f"\nCoefficients:")
print(f"  Intercept:   {slm_ml.betas[0][0]:10.5f}")
print(f"  EDMDIAWN3M:  {slm_ml.betas[1][0]:10.5f}")
print(f"  WSSRCEHIMP:  {slm_ml.betas[2][0]:10.5f}")
print(f"  AHBTHPWIMG:  {slm_ml.betas[3][0]:10.5f}")
print(f"  EDEDUCWSEH:  {slm_ml.betas[4][0]:10.5f}")
print(f"\nSpatial parameter:")
print(f"  Rho (ρ):     {slm_ml.rho:10.5f}")
print(f"\nModel fit:")
print(f"  Log-likelihood: {slm_ml.logll:.4f}")
k = len(slm_ml.betas) + 1
aic = -2 * slm_ml.logll + 2 * k
print(f"  AIC: {aic:.2f}")
print(f"  N observations: {slm_ml.n}")


SPATIAL LAG MODEL RESULTS (PYTHON - ML)

Coefficients:
  Intercept:     28.00114
  EDMDIAWN3M:    -0.41446
  WSSRCEHIMP:     0.26574
  AHBTHPWIMG:    -0.16426
  EDEDUCWSEH:    -0.35008

Spatial parameter:
  Rho (ρ):        0.37791

Model fit:
  Log-likelihood: -100.4223
  AIC: 214.84
  N observations: 31


### 10) Residual Moran's I after SAR lag

In [16]:
# Get SAR lag residuals
residuals_slm = slm_ml.u.flatten()

# Moran's I test on SLM residuals
moran_slm_resid = Moran(residuals_slm, w_queen)

print("="*60)
print("MORAN'S I ON SPATIAL LAG MODEL RESIDUALS (PYTHON)")
print("="*60)
print(f"Moran's I: {moran_slm_resid.I:.6f}")
print(f"Expected I: {moran_slm_resid.EI:.6f}")
print(f"Z-score: {moran_slm_resid.z_norm:.6f}")
print(f"P-value: {moran_slm_resid.p_norm:.6f}")
print("\nInterpretation: Check if spatial autocorrelation in residuals")
print("has been reduced compared to OLS.")
print("="*60)

# Compare Moran's I for OLS vs SLM residuals
print("\n" + "="*60)
print("RESIDUAL AUTOCORRELATION COMPARISON")
print("="*60)
print(f"OLS residuals - Moran's I: {moran_resid.I:.6f} (p={moran_resid.p_norm:.6f})")
print(f"SLM residuals - Moran's I: {moran_slm_resid.I:.6f} (p={moran_slm_resid.p_norm:.6f})")
print(f"\nReduction in autocorrelation: {(moran_resid.I - moran_slm_resid.I):.6f}")
print("="*60)

MORAN'S I ON SPATIAL LAG MODEL RESIDUALS (PYTHON)
Moran's I: 0.002619
Expected I: -0.033333
Z-score: 0.263907
P-value: 0.791852

Interpretation: Check if spatial autocorrelation in residuals
has been reduced compared to OLS.

RESIDUAL AUTOCORRELATION COMPARISON
OLS residuals - Moran's I: 0.226981 (p=0.056023)
SLM residuals - Moran's I: 0.002619 (p=0.791852)

Reduction in autocorrelation: 0.224362


### 11) Compute direct, indirect and total impacts

In spatial lag models, the effect of a change in an explanatory variable has three components:

- **Direct impact**: Effect on the observation itself
- **Indirect impact**: Spillover effects on neighboring observations  
- **Total impact**: Sum of direct and indirect effects

In [17]:
# Compute impacts for spatial lag model
# Direct, indirect, and total effects

# Get coefficients (excluding intercept)
betas = slm_ml.betas[1:]  # Exclude intercept
rho = slm_ml.rho
n = slm_ml.n

# Compute (I - ρW)^(-1)
I = np.eye(n)
W = w_queen.sparse.toarray()
inv_matrix = np.linalg.inv(I - rho * W)

print("="*60)
print("SPATIAL LAG MODEL - IMPACTS")
print("="*60)

for idx, var_name in enumerate(vars_x):
    beta = betas[idx][0]

    # Direct effect: average of diagonal elements
    direct = np.mean(np.diag(inv_matrix)) * beta

    # Total effect: sum of all elements / n
    total = np.mean(inv_matrix) * beta * n

    # Indirect effect: total - direct
    indirect = total - direct

    print(f"\n{var_name}:")
    print(f"  Direct:   {direct:10.5f}")
    print(f"  Indirect: {indirect:10.5f}")
    print(f"  Total:    {total:10.5f}")

print("\n" + "="*60)
print("Interpretation:")
print("- Direct: Average effect on the unit itself")
print("- Indirect: Average spillover effect on all other units")
print("- Total: Overall average effect including feedback loops")
print("="*60)

SPATIAL LAG MODEL - IMPACTS

EDMDIAWN3M:
  Direct:     -0.43651
  Indirect:   -0.22973
  Total:      -0.66624

WSSRCEHIMP:
  Direct:      0.27987
  Indirect:    0.14730
  Total:       0.42716

AHBTHPWIMG:
  Direct:     -0.17300
  Indirect:   -0.09105
  Total:      -0.26404

EDEDUCWSEH:
  Direct:     -0.36870
  Indirect:   -0.19404
  Total:      -0.56274

Interpretation:
- Direct: Average effect on the unit itself
- Indirect: Average spillover effect on all other units
- Total: Overall average effect including feedback loops
